In [5]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

# Create an instance of the OpenAI class
client = OpenAI(api_key=openai_api_key)

assistant = client.beta.assistants.create(
    name="Python Data Analyst",
    instructions="You are a data analyst working with a Python data analysis library. Write code, analyze data to provide insights.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o"
)

In [6]:
thread = client.beta.threads.create()

In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to calculate the mean of a list of numbers: [1,2,3,5,3,2,2]."
)

In [8]:
from typing_extensions import override
from openai import AssistantEventHandler

# Create an event handler class to define how to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nAssistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot) -> None:
        print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call) -> None:
        print(f"\nAssistant > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot) -> None:
        if delta.type == "code_interpreter":
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

In [9]:
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Help the user with his data analysis task.",
    event_handler=EventHandler()
) as stream:
    stream.until_done()


Assistant > code_interpreter

# Calculate the mean of the given list of numbers
numbers = [1, 2, 3, 5, 3, 2, 2]
mean_value = sum(numbers) / len(numbers)
mean_value
Assistant > The mean of the list of numbers \([1, 2, 3, 5, 3, 2, 2]\) is approximately \(2.57\).

In [10]:
thread

Thread(id='thread_8T7FElINmUOt7tp9MeVcniS1', created_at=1741179209, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))